# MCT4052 Workshop 6a: Storing and Reloading Features with Pandas

*Author: Stefano Fasciani, stefano.fasciani@imv.uio.no, Department of Musicology, University of Oslo.*

Computing features from files (i.e. raw-data) can be a time consuming task. It may be convenient to compute the features once for all and store them in a file, together with the associated label (numeric and/or text) and aditional metadata, such as the  filename. Then we can simply reload computed features from a file, which is significantly faster. We can also compute and store in a file a much larger set of features than what we will actually decide to use later on.

In [1]:
import numpy as np
import pandas as pd
import librosa, librosa.display
import matplotlib.pyplot as plt
import matplotlib.style as ms
ms.use('seaborn-muted')
import IPython.display as Ipd
import os
import sklearn

In [2]:
sr = 22050

def extract_features(filename, sr):
    
    signal, dummy = librosa.load(filename, sr, mono=True)
    
    output = [
        np.mean(librosa.feature.zero_crossing_rate(signal)),
        np.mean(librosa.feature.spectral_centroid(signal)),
        np.mean(librosa.feature.spectral_contrast(signal)), 
        np.mean(librosa.feature.spectral_flatness(signal)),
        np.mean(librosa.feature.spectral_bandwidth(signal)),
        np.mean(librosa.feature.spectral_rolloff(signal)),
        np.mean(librosa.feature.rms(signal))
    ]
    
    return output


#creating an array of zeros of the proper size where we will store computed features and lables
filenames = os.listdir('./data/examples2')
features = np.zeros((len(filenames),7))
labels = np.zeros((len(filenames))) #we store the labels as integers
labels_text = [] #empty list
classes = ['kick','snare','cymbal','clap'] #list of text labels to conver numberical labels to text labels

for i in range(len(filenames)):
    #print('processing',filenames[i])
    features[i,:] = extract_features('./data/examples2/'+filenames[i], sr)
    if filenames[i].find('kick') != -1:
        labels[i] = 0
    elif filenames[i].find('snare') != -1:
        labels[i] = 1
    elif filenames[i].find('cymbal') != -1:
        labels[i] = 2
    elif filenames[i].find('clap') != -1:
        labels[i] = 3
        
    labels_text.append(classes[int(labels[i])])

print('Done!')

Done!


### 1. Merging arrays in a data structure and saving to a .csv file

In this example we use text tags/labels to address the columns which will be useful when retrieving the features later on.

In [3]:
#merging everything into a single data structure
dataset = pd.DataFrame(features)
dataset.columns = ['zcr','cent','cont','flat','band','roll','rms'] #naming the features, this is optional, and tedious when you have many
dataset['label digit'] = labels #creating column for labels both as integer digit and text (one would be sufficient)
dataset['label text'] = labels_text 
dataset['filename'] = filenames

#we can save the dataset to a file to resume working without re-computing the features (unless you want to change them)
dataset.to_csv('dataset.csv')

#displaying the data structure
dataset

,zcr,cent,cont,flat,band,roll,rms,label digit,label text,filename
0,0.169027,3667.132840,16.857274,0.119845,2853.236527,6971.374512,0.246108,1.0,snare,snare_28.wav
1,0.190283,3527.353876,18.688378,0.068258,2874.171495,6918.618164,0.129410,1.0,snare,snare_00.wav
2,0.178861,3193.150255,18.326836,0.066239,2394.130624,5780.836839,0.157899,1.0,snare,snare_14.wav
3,0.195374,3476.882658,17.851301,0.106591,2775.355411,6768.154907,0.106924,1.0,snare,snare_15.wav
4,0.208350,4022.987666,17.339755,0.094670,3058.699835,7665.820312,0.041672,1.0,snare,snare_01.wav
...,...,...,...,...,...,...,...,...,...,...
162,0.147081,2933.023072,17.272802,0.052344,2453.666923,5580.688477,0.167242,1.0,snare,snare_18.wav
163,0.222860,3236.297921,17.178865,0.103735,2446.074302,6083.129883,0.112644,1.0,snare,snare_19.wav
164,0.144420,3339.514468,18.870118,0.053724,2626.781881,6262.736373,0.086612,1.0,snare,snare_31.wav
165,0.186179,3225.368167,18.111396,0.053382,2267.723325,5707.565487,0.171652,1.0,snare,snare_25.wav


### 2. Loading data from .csv file to a data structure and extracting data in separate arrays

In [4]:
#no we reload from file and re-extract the three separate arrays containing features, labels, and filenames
imported_dataset = pd.read_csv('dataset.csv')
imported_features = dataset[['zcr','cent','cont','flat','band','roll','rms']].to_numpy() #here we can go by name or column number, by name we can use any order we prefer
imported_labels = dataset['label digit'] #we keep this as a pandas data structure, it will be handy to trace back misclassified files (because it's indexed)
imported_labels_text = dataset['label text']
imported_filenames = dataset['filename'].tolist()
print(imported_labels)
print(imported_labels_text)
print(imported_filenames)
print(imported_features)

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
162    1.0
163    1.0
164    1.0
165    1.0
166    0.0
Name: label digit, Length: 167, dtype: float64
0      snare
1      snare
2      snare
3      snare
4      snare
       ...  
162    snare
163    snare
164    snare
165    snare
166     kick
Name: label text, Length: 167, dtype: object
['snare_28.wav', 'snare_00.wav', 'snare_14.wav', 'snare_15.wav', 'snare_01.wav', 'snare_29.wav', 'snare_17.wav', 'snare_03.wav', 'snare_02.wav', 'snare_16.wav', 'snare_12.wav', 'snare_06.wav', 'clap41.wav', 'clap40.wav', 'snare_07.wav', 'snare_13.wav', 'snare_05.wav', 'snare_11.wav', 'snare_39.wav', 'snare_38.wav', 'snare_10.wav', 'snare_04.wav', 'kick_07.wav', 'kick_13.wav', 'clap30.wav', 'clap24.wav', 'clap18.wav', 'cymbal29.wav', 'cymbal01.wav', 'cymbal15.wav', 'cymbal14.wav', 'cymbal00.wav', 'cymbal28.wav', 'clap19.wav', 'clap25.wav', 'clap31.wav', 'kick_12.wav', 'kick_06.wav', 'kick_10.wav', 'kick_04.wav', 'kick_38.wav', 'clap27.wa

### 3. Same example as above but without using column names
When working with a large number of features entering a text name for each column is time consuming, it's easier to work with the column number. Mind that in this case we must track in which columns the features were stored.

In [5]:
#merging everything into a single data structure
dataset = pd.DataFrame(features)
dataset['label digit'] = labels #creating column for labels both as integer digit and text (one would be sufficient)
dataset['label text'] = labels_text 
dataset['filename'] = filenames

#we can save the dataset to a file to resume working without re-computing the features (unless you want to change them)
dataset.to_csv('dataset.csv')

#displaying the data structure
dataset

,0,1,2,3,4,5,6,label digit,label text,filename
0,0.169027,3667.132840,16.857274,0.119845,2853.236527,6971.374512,0.246108,1.0,snare,snare_28.wav
1,0.190283,3527.353876,18.688378,0.068258,2874.171495,6918.618164,0.129410,1.0,snare,snare_00.wav
2,0.178861,3193.150255,18.326836,0.066239,2394.130624,5780.836839,0.157899,1.0,snare,snare_14.wav
3,0.195374,3476.882658,17.851301,0.106591,2775.355411,6768.154907,0.106924,1.0,snare,snare_15.wav
4,0.208350,4022.987666,17.339755,0.094670,3058.699835,7665.820312,0.041672,1.0,snare,snare_01.wav
...,...,...,...,...,...,...,...,...,...,...
162,0.147081,2933.023072,17.272802,0.052344,2453.666923,5580.688477,0.167242,1.0,snare,snare_18.wav
163,0.222860,3236.297921,17.178865,0.103735,2446.074302,6083.129883,0.112644,1.0,snare,snare_19.wav
164,0.144420,3339.514468,18.870118,0.053724,2626.781881,6262.736373,0.086612,1.0,snare,snare_31.wav
165,0.186179,3225.368167,18.111396,0.053382,2267.723325,5707.565487,0.171652,1.0,snare,snare_25.wav


In [6]:
#no we reload from file and re-extract the three separate arrays containing features, labels, and filenames
imported_dataset = pd.read_csv('dataset.csv')

#we stored 7 features when the data structure was empty, so they eneded in culumns 0 to 6 (included)
#iloc[:,0:6] means : ==> all rows, 0:7 ==> columns 0 to 6 (7 is not included)
imported_features = dataset.iloc[:,0:7].to_numpy()#we stored 7 features when the data structure was empty, so they eneded in culumns 0 to 6 
imported_labels = dataset['label digit'] #we keep this as a pandas data structure, it will be handy to trace back misclassified files (because it's indexed)
imported_labels_text = dataset['label text']
imported_filenames = dataset['filename'].tolist()
print(imported_labels)
print(imported_labels_text)
print(imported_filenames)
print(imported_features)

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
162    1.0
163    1.0
164    1.0
165    1.0
166    0.0
Name: label digit, Length: 167, dtype: float64
0      snare
1      snare
2      snare
3      snare
4      snare
       ...  
162    snare
163    snare
164    snare
165    snare
166     kick
Name: label text, Length: 167, dtype: object
['snare_28.wav', 'snare_00.wav', 'snare_14.wav', 'snare_15.wav', 'snare_01.wav', 'snare_29.wav', 'snare_17.wav', 'snare_03.wav', 'snare_02.wav', 'snare_16.wav', 'snare_12.wav', 'snare_06.wav', 'clap41.wav', 'clap40.wav', 'snare_07.wav', 'snare_13.wav', 'snare_05.wav', 'snare_11.wav', 'snare_39.wav', 'snare_38.wav', 'snare_10.wav', 'snare_04.wav', 'kick_07.wav', 'kick_13.wav', 'clap30.wav', 'clap24.wav', 'clap18.wav', 'cymbal29.wav', 'cymbal01.wav', 'cymbal15.wav', 'cymbal14.wav', 'cymbal00.wav', 'cymbal28.wav', 'clap19.wav', 'clap25.wav', 'clap31.wav', 'kick_12.wav', 'kick_06.wav', 'kick_10.wav', 'kick_04.wav', 'kick_38.wav', 'clap27.wa